In [1]:
!pip install ultralytics opencv-python pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from scipy.spatial import distance

# Load YOLOv8 nano model
model = YOLO("yolov8n.pt")

# Open video file
video_path = "/content/dataset_video.mp4"  # Change to your video file
cap = cv2.VideoCapture(video_path)

# Parameters
crowd_threshold = 3  # Minimum persons to be considered a crowd
distance_threshold = 50  # Pixel distance to define "close" people
frames_required = 10  # Number of consecutive frames for crowd detection

# Tracking history
frame_crowd_count = {}  # Dictionary to store crowd persistence

# CSV logging
csv_data = []

frame_no = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit if video ends

    frame_no += 1

    # Run YOLOv8 inference
    results = model(frame)
    persons = []

    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            if int(cls) == 0:  # Class 0 is 'person' in YOLOv8
                cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)  # Get centroid
                persons.append((cx, cy))

    # Check for crowd formation
    if len(persons) >= crowd_threshold:
        close_groups = []
        for i, p1 in enumerate(persons):
            close_group = [p1]
            for j, p2 in enumerate(persons):
                if i != j and distance.euclidean(p1, p2) < distance_threshold:
                    close_group.append(p2)

            if len(close_group) >= crowd_threshold:
                close_groups.append(close_group)

        # If a crowd is detected, update the persistence dictionary
        if close_groups:
            frame_crowd_count[frame_no] = len(close_groups[0])  # Track first detected crowd

    # Remove old frame entries
    expired_frames = [key for key in frame_crowd_count if frame_no - key > frames_required]
    for key in expired_frames:
        del frame_crowd_count[key]

    # If a group is consistently detected for `frames_required` frames, log it
    for start_frame, count in frame_crowd_count.items():
        if frame_no - start_frame == frames_required:
            csv_data.append([start_frame, count])
            print(f"Crowd detected: Frame {start_frame}, Persons in Crowd: {count}")

cap.release()

# Save results to CSV
df = pd.DataFrame(csv_data, columns=["Frame Number", "Person Count in Crowd"])
df.to_csv("crowd_detection_log.csv", index=False)

print("Crowd detection completed. Results saved to crowd_detection_log.csv")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 71.4MB/s]



0: 384x640 37 persons, 2 birds, 367.1ms
Speed: 27.5ms preprocess, 367.1ms inference, 38.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 3 birds, 161.9ms
Speed: 6.0ms preprocess, 161.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 3 birds, 172.3ms
Speed: 5.5ms preprocess, 172.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 3 birds, 157.0ms
Speed: 5.9ms preprocess, 157.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 2 birds, 160.3ms
Speed: 5.8ms preprocess, 160.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 2 birds, 144.6ms
Speed: 4.5ms preprocess, 144.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 2 birds, 180.3ms
Speed: 4.5ms preprocess, 180.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 2